In [1]:
!pip install gradio transformers bitsandbytes accelerate peft trl pandas datasets

In [2]:
import gradio as gr
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer
from threading import Thread
import pandas as pd
import bitsandbytes

In [3]:
!pip install --upgrade pandas datasets transformers

import torch
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

# Configuration
dataset_id = "jojogo9/Food_Recipes"
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
output_model = "llm-007"
start_row = 150  # Starting row
end_row = 250  # Ending row
sample_size = 100  # Sample size for the dataset
max_length = 128  # Max length for tokenization

# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# !pip install --upgrade pandas datasets transformers

# import torch
# from datasets import load_dataset, Dataset
# from transformers import AutoModelForCausalLM, AutoTokenizer

# # Configuration
# dataset_id = "jojogo9/Food_Recipes"
# model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
# output_model = "llm-007"
# start_row = 150  # Starting row
# end_row = 250  # Ending row
# sample_size = 100  # Sample size for the dataset
# max_length = 128  # Max length for tokenization

# # Check device
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Function to prepare training data with start and end row parameters
def prepare_train_data(data_id, start_row=0, end_row=None, sample_size=200):
    # Load the dataset
    data = load_dataset(data_id, split="train")
    data_df = data.to_pandas()

    # Select the rows between start_row and end_row
    if end_row is not None:
        data_df = data_df.iloc[start_row:end_row]
    else:
        data_df = data_df.iloc[start_row:]

    # If the resulting dataset is larger than sample_size, sample the data
    if len(data_df) > sample_size:
        data_df = data_df.sample(n=sample_size, random_state=42)

    # Format the text column as required
    data_df["text"] = data_df.apply(
        lambda x: (
            "im_start user\n" + x["ingredients"] + "\nim_end\n" +
            "im_start assistant\n" + x["name"] + "\nim_end\n" +
            "im_start id: " + str(x["id"]) + "\nim_end\n" +
            "im_start steps: " + x["steps"] + "\nim_end\n"
        ),
        axis=1
    )

    # Convert the DataFrame back to a Dataset
    data = Dataset.from_pandas(data_df)

    return data

# Prepare the training data
data = prepare_train_data(dataset_id, start_row=start_row, end_row=end_row, sample_size=sample_size)

# Function to get model and tokenizer
def get_model_and_tokenizer(model_id):
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        # load_in_8bit=True  # If 4-bit isn't supported directly
    )
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer

# Load model and tokenizer
model, tokenizer = get_model_and_tokenizer(model_id)

# Function to tokenize, encode, and truncate the dataset
def encode_and_truncate_dataset(dataset, tokenizer, max_length=128):
    def tokenize_function(examples):
        return tokenizer(
            examples["text"],
            padding="max_length",  # Pad to the maximum length
            truncation=True,       # Truncate to the maximum length
            max_length=max_length  # Maximum length of sequences
        )

    tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])
    return tokenized_dataset

# Encode and truncate the dataset
encoded_dataset = encode_and_truncate_dataset(data, tokenizer, max_length)

# Example to check the first encoded item
print(encoded_dataset[0])

# Save the dataset if needed
encoded_dataset.save_to_disk("encoded_dataset")

# Now your encoded_dataset is ready for training with the language model


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/231637 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

{'name': 'bake in  oozy bread', 'id': 209203, 'minutes': 35, 'contributor_id': 86764, 'submitted': '2007-02-04', 'tags': "['bacon', '60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', '5-ingredients-or-less', 'breads', 'breakfast', 'pork', 'american', 'southern-united-states', 'easy', 'kid-friendly', 'dietary', 'meat', 'brunch', 'presentation', 'served-hot']", 'nutrition': '[1063.2, 126.0, 155.0, 74.0, 34.0, 168.0, 21.0]', 'n_steps': 14, 'steps': '[\'preheat oven to 375f\', \'place a medium skillet over medium-high heat and add bacon\', \'cook until crisp , about 8-10 minutes and reserve\', \'place a small saucepot over medium heat and add the maple syrup and the butter -- do not bring up to a boil\', \'cook until butter is melted , then turn the heat off and reserve\', \'butter or spray a 3x9-inch loaf pan\', \'pour a couple tablespoons of the maple / butter sauce in the bottom\', \'layer 3 of the biscuits in the b

Saving the dataset (0/1 shards):   0%|          | 0/100 [00:00<?, ? examples/s]

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer

# Define your LoraConfig
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Define a function to get model and tokenizer
def get_model_and_tokenizer(model_id):
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        # Remove quantization_config argument
    )
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.pad_token = tokenizer.eos_token
    return model, tokenizer

# Example usage
model_id = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'
model, tokenizer = get_model_and_tokenizer(model_id)


In [7]:

# Define the training arguments
training_arguments = TrainingArguments(
    output_dir="output_model",
    per_device_train_batch_size=2,  # Reduce batch size 8
    gradient_accumulation_steps=32,   # Increase accumulation steps 8
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    logging_steps=10,
    num_train_epochs=3,
    max_steps=100, #PUT 250 LATER
    fp16=True,
    # push_to_hub=True
)

# Assume 'data' is your training dataset
trainer = SFTTrainer(
    model=model,
    train_dataset=data,
    peft_config=peft_config,
    dataset_text_field="text",
    args=training_arguments,
    tokenizer=tokenizer,
    packing=False,
    max_seq_length=1024
)


trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
10,1.926800
20,1.629000
30,1.447800
40,1.369700
50,1.308200
60,1.264900
70,1.232600
80,1.229200
90,1.219000
100,1.216700


TrainOutput(global_step=100, training_loss=1.3843943881988525, metrics={'train_runtime': 902.7765, 'train_samples_per_second': 7.089, 'train_steps_per_second': 0.111, 'total_flos': 1.3412607602245632e+16, 'train_loss': 1.3843943881988525, 'epoch': 64.0})

In [8]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2048, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=2048, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): lora.Linear(
            (base_layer): Linear(in_features=2048, out_features=256,

In [9]:
trainer.evaluate(encoded_dataset)

{'eval_loss': 1.0757161378860474,
 'eval_runtime': 2.1301,
 'eval_samples_per_second': 46.946,
 'eval_steps_per_second': 6.103,
 'epoch': 64.0}

In [10]:
from peft import AutoPeftModelForCausalLM, PeftModel
from transformers import AutoModelForCausalLM
import torch

import os

model = AutoModelForCausalLM.from_pretrained(model_id,torch_dtype=torch.float16,load_in_8bit=False,
                                             device_map="auto",
                                             trust_remote_code=True)
model_path = "/content/output_model/checkpoint-100"
# model_path = "/recipe-generator-G97"

peft_model = PeftModel.from_pretrained(model,model_path,from_transformers=True,device_map="auto")
model = peft_model.merge_and_unload()

In [11]:
!pip install huggingface_hub --q
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [13]:
model.push_to_hub("eaglet/JulyTesting")
tokenizer.push_to_hub("eaglet/JulyTesting")

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/eaglet/JulyTesting/commit/2b675978b7b8ad8697d6010216129caf17349983', commit_message='Upload tokenizer', commit_description='', oid='2b675978b7b8ad8697d6010216129caf17349983', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from huggingface_hub import HfApi, HfFolder, Repository

# Replace with your Hugging Face username and model name
username = "nithin"
model_name = "finetuned-llama"

# Initialize the API
api = HfApi()

# Create a repository
repo_url = api.create_repo(repo_id=f"{username}/{model_name}", private=False)

# Clone the repository
repo = Repository(local_dir="finetuned-llama", clone_from=repo_url)

# Add files to the repository
repo.git_add(auto_lfs_track=True)
repo.git_commit("Initial commit of the fine-tuned model")
repo.git_push()


In [ ]:
from transformers import GenerationConfig
from time import perf_counter

def generate_resp(user_input):
  prompt = formated_prompt(user_input)
  inputs = tokenizer([prompt], return_tensors="pt")
  generation_config = GenerationConfig(penalty_alpha=0.6,do_sample=True,
                                         max_new_tokens=100,pad_token_id=tokenizer.eos_token_id)
  start_time = perf_counter()
  inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(**inputs,generation_config=generation_config)
  # print(tokenizer.decode(outputs[0],skip_special_tokens=True))
  output_time = perf_counter() - start_time
  print("Time taken ",round(output_time,2))
  return tokenizer.decode(outputs[0],skip_special_tokens=True)

In [ ]:
generate_resp(user_input="make a cake")